# `Regression` - 新手实践神经网络

## 0. 导入包

In [98]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

## 1. 数据处理

用 `pandas` 的 `read` 方法读入 `.csv` 文件，后可用 `.values` 将内容转为 `numpy.ndarray` 返回

In [99]:
# train_data size: 2699 x 118 (id + 37 states + 16 features x 5 days)
# test_data size: 1078 x 117 (without last day's positive rate)
train_data, test_data = pd.read_csv('./covid.train.csv').values, pd.read_csv('./covid.test.csv').values

可将训练集分为 `train` 集和 `valid` 集
- 我们用一个函数将 `train_data` 切分：我们需要定义一个函数 
  ```python
  train_valid_split(numpy.ndarray: train_data, float: valid_ratio, int: seed) -> (numpy.ndarray, numpy.ndarray)
  ```
- 同时为了我们的参数 `valid_ratio, seed` 方便设置，我们将所有 `hyperparameter` 存在字典 `config` 中
- 为了实现可复现性，保证各个随机数生成器的 `seed` 相同，我们定义函数 `same_seed` 将可用到的随机数生成种子设为相同

In [100]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    # 如果当前系统支持 CUDA，设置使用 CuDNN 库的随机数生成器的种子，以确保结果可重复。
    torch.backends.cudnn.deterministic = True
    # 如果当前系统支持 CUDA，禁用使用 CuDNN 库的自动调优功能，以确保结果可重复。
    torch.backends.cudnn.benchmark = False
    # 设置 NumPy 随机数生成器的种子。
    np.random.seed(seed)
    # 设置 PyTorch 随机数生成器的种子
    torch.manual_seed(seed)
    if torch.cuda.is_available():   # 如果当前系统支持 CUDA，设置所有可用的 CUDA 设备的随机数生成器的种子。
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set))
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 6000,     # Number of epochs.
    'select_all': False,   # Whether to use all features.
    'batch_size': 216,
    'learning_rate': 1e-5,
    'early_stop': 400,    # If model has not improved for this many consecutive epochs, stop training.
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

same_seed(config['seed'])

train_data = pd.read_csv('./covid.train.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape}
valid_data size: {valid_data.shape}
test_data size: {test_data.shape}""")


train_data size: (2160, 118)
valid_data size: (539, 118)
test_data size: (1078, 117)


对于 `feature` 我们不一定要用上所有，可以选取一部分使用
- 需要定义函数 `select_feature` 来选取要进行训练的 `feature`，并将 `feature` 和 `label` 分开

In [101]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    y_train, y_valid = train_data[:, -1], valid_data[:, -1]
    x_raw_train, x_raw_valid, x_raw_test= train_data[:, :-1], valid_data[:, :-1], test_data

    if select_all:
        select_index =  list(x_raw_train.shape[1])
    else:
        select_index = list(range(38, 117))

    return x_raw_train[:, select_index], x_raw_valid[:, select_index], x_raw_test[:, select_index], y_train, y_valid

x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])
print(f'number of features: {x_train.shape[1]}')

number of features: 79


定义自己的 `Dataset`，并创建 `Dataset` 实例

> A custom Dataset `class` must implement three functions: `__init__`, `__len__`, and `__getitem__`.
> - The `__len__` function returns the number of samples in our dataset.
> - The `__getitem__` function loads and returns a sample from the dataset at the given index idx.

In [102]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                             COVID19Dataset(x_valid, y_valid), \
                                             COVID19Dataset(x_test)

用 `DataLoader` 加载 `Dataset`它可以将数据集分批次加载到内存中，并对每个批次进行预处理和数据增强等操作。
> The Dataset retrieves our dataset’s features and labels one sample at a time. While training a model, we typically want to pass samples in “minibatches”, reshuffle the data at every epoch to reduce model overfitting, and use Python’s multiprocessing to speed up data retrieval.

In [103]:
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

## 构建模型

We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__. Every nn.Module subclass implements the operations on input data in the forward method.

In [107]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 79),
            nn.BatchNorm1d(79),
            nn.ReLU(),
            nn.Dropout(p=0.35),
            nn.Linear(79, 1)
        )

    def forward(self, x):
        # 前向传播逻辑
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x


定义 `training loop`

In [108]:
def trainer(train_loader, valid_loader, model, config, device):
    # Loss Function
    criterion = nn.MSELoss(reduction='mean')
    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device.
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())

            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

## 开始训练

In [ ]:
model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

## 可视化训练曲线

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

## 测试模型

测试并保存数据

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu())
    preds = torch.cat(preds, dim=0).numpy()
    return preds

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
save_pred(preds, 'pred.csv')